In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from dataclasses import dataclass

torch.manual_seed(1024)

In [ ]:
@dataclass
class CPTConfig:
    block_size: int = 512
    batch_size: int = 12
    n_layer: int = 12
    n_head: int = 12
    n_embed: int = 768 # hidden_dim, hidden_size (tie_embedding_weight)
    hidden_dim: int = n_embed
    dropout: float = 0.1
    head_size: int = n_embd // n_head
    vocab_size: int = 50207


In [ ]:
class SingleHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.key = nn.Linear(config.)